In [14]:
# Imports de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns      
import missingno as msno
import statsmodels.api as sm
import matplotlib.dates as mdates

from apafib import load_dormir
from scipy import stats
from time import time
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy


import warnings
warnings.filterwarnings('ignore')

# Definiciones
RND = 16
MAX_ITER = 5000

sns.set(style="whitegrid", font_scale=1.05)

# Funciones auxiliares
def format_pval(p):
    if p == 0:
        return r"$0$"
    exp = int(np.floor(np.log10(p)))
    if exp >= -3:
        return f"{p:.4f}"                
    else:
        mant = p / (10.0**exp)
        return rf"${mant:.2f}\times10^{{{exp}}}$"
    
# quitar warnings:
import warnings
warnings.filterwarnings("ignore", message="findfont:.*")

init_time = time()

# Práctica de APA: Tennis. EXTRA.

## Introducción

Autores: Oriol Farrés y Marc Gil

## 1. El Desafío: Benchmarking contra IBM. Predicciones Roland Garross 2024

El objetivo es medir la capacidad predictiva de nuestro **Stacking Ensemble** frente a **IBM Watson**, el predictor oficial de Roland Garros. No solo buscamos un buen resultado, sino validar si un modelo desarrollado académicamente puede competir con una infraestructura profesional en el torneo de tierra batida más exigente del mundo.

## 2. Metodología: Validación Walk-Forward (Ronda a Ronda)

Para garantizar el máximo realismo y evitar sesgos, implementaremos una estrategia de **entrenamiento dinámico**:

* **División Temporal:** El torneo se segmenta en sus 7 rondas oficiales (desde R128 hasta la Final).
* **Entrenamiento "al día":** Para predecir cada ronda, el modelo se entrena con todos los datos históricos (2011-2024) más los resultados de las rondas ya finalizadas de ese mismo torneo.
* **Simulación Real:** Este enfoque garantiza que, al predecir la final, el modelo ya haya procesado el estado de forma y fatiga mostrado por los finalistas en sus 6 partidos previos en París.

---


## Estructura de la práctica





In [15]:
ibm = pd.read_parquet('./data/clean/ibm.parquet')
ibm.head()

,surface,draw_size,tourney_level,tourney_date,p1_entry,p1_hand,p2_entry,p2_hand,best_of,tourney_points,...,diff_1st_won_pct_last_1,diff_1st_won_pct_last_10,diff_elo,diff_elo_surface,year,month,day,tourney_name,round,round_order
0,Clay,128,G,2012-05-27,DA,R,DA,R,5,2000,...,-0.009081,0.008778,113.831337,-36.202833,2012,5,27,Roland Garros,R128,4
1,Clay,32,A,2012-02-20,DA,R,LL,L,3,250,...,0.116450,0.167977,97.367302,29.751678,2012,2,20,Buenos Aires,R32,6
2,Hard,128,M,2016-03-21,DA,R,DA,R,3,1000,...,0.135052,-0.075319,-98.844854,-57.555792,2016,3,21,Miami Masters,R128,4
3,Hard,32,A,2017-02-27,DA,R,DA,R,3,500,...,0.014713,-0.022251,-29.749371,-57.929474,2017,2,27,Acapulco,R32,6
4,Hard,64,M,2017-10-09,DA,R,DA,L,3,1000,...,0.130125,-0.008239,-122.641670,18.301428,2017,10,9,Shanghai Masters,R64,5


---

## División de las 7 rondas

In [16]:
import os
import pandas as pd

# 1. Preparación de rutas y datos
output_dir = "data/ibm"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Columnas de ruido a eliminar antes de guardar (solo sirven para filtrado)
cols_ruido = ['tourney_name', 'round', 'round_order', 'year', 'month', 'day', 'tourney_date']

# Asegurar que las fechas y el orden de rondas sean correctos
ibm['tourney_date'] = pd.to_datetime(ibm['tourney_date'])
rg_2024 = ibm[(ibm['tourney_name'].str.contains('Roland Garros')) & (ibm['year'] == 2024)].copy()

# Rondas oficiales de un Grand Slam (R128 a Final)
rondas_ids = [4, 5, 6, 7, 8, 9, 10]
nombres_rondas = ['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'F']

print(f"--- Iniciando división física de {len(rondas_ids)} rondas ---")
print(f"⚠️  Eliminando columnas de ruido: {cols_ruido}\n")

# 2. Bucle de generación de archivos
for i, r_id in enumerate(rondas_ids):
    nombre = nombres_rondas[i]
    
    # Identificar la fecha de inicio de la ronda actual
    fecha_inicio_ronda = rg_2024[rg_2024['round_order'] == r_id]['tourney_date'].min()
    
    # Conjunto de Entrenamiento: Todo lo anterior a esta ronda
    df_train = ibm[ibm['tourney_date'] < fecha_inicio_ronda].copy()
    
    # Conjunto de Validación: Solo los partidos de esta ronda
    df_val = rg_2024[rg_2024['round_order'] == r_id].copy()
    
    # Eliminar columnas de ruido antes de guardar
    df_train = df_train.drop(columns=[c for c in cols_ruido if c in df_train.columns])
    df_val = df_val.drop(columns=[c for c in cols_ruido if c in df_val.columns])
    
    # Guardar archivos físicos
    train_path = f"{output_dir}/train_step_{i+1}_{nombre}.parquet"
    val_path = f"{output_dir}/val_step_{i+1}_{nombre}.parquet"
    
    df_train.to_parquet(train_path, index=False)
    df_val.to_parquet(val_path, index=False)
    
    print(f"Paso {i+1} ({nombre}): Train -> {df_train.shape[0]} filas, {df_train.shape[1]} cols | Val -> {df_val.shape[0]} filas, {df_val.shape[1]} cols")

print(f"\n✅ Archivos guardados en {output_dir} (sin columnas de ruido)")

--- Iniciando división física de 7 rondas ---
⚠️  Eliminando columnas de ruido: ['tourney_name', 'round', 'round_order', 'year', 'month', 'day', 'tourney_date']

Paso 1 (R128): Train -> 34519 filas, 49 cols | Val -> 64 filas, 49 cols
Paso 2 (R64): Train -> 34519 filas, 49 cols | Val -> 29 filas, 49 cols
Paso 3 (R32): Train -> 34519 filas, 49 cols | Val -> 16 filas, 49 cols
Paso 4 (R16): Train -> 34519 filas, 49 cols | Val -> 8 filas, 49 cols
Paso 5 (QF): Train -> 34519 filas, 49 cols | Val -> 3 filas, 49 cols
Paso 6 (SF): Train -> 34519 filas, 49 cols | Val -> 2 filas, 49 cols
Paso 7 (F): Train -> 34519 filas, 49 cols | Val -> 1 filas, 49 cols

✅ Archivos guardados en data/ibm (sin columnas de ruido)


In [17]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import os

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn import set_config

set_config(transform_output="pandas")

# Definimos target
target = 'Winner'

features = ibm.columns.drop(target).tolist()
numerical_features = ibm.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = ibm.select_dtypes(include=['object', 'category']).columns.tolist()

# Eliminar el target de las númericas (está codificada ya como 0/1)
numerical_features.remove(target)

print("--- Clasificación de Variables ---")
print(f"Target: {target}")
print(f"Numéricas ({len(numerical_features)}): {numerical_features}")
print(f"Categóricas ({len(categorical_features)}): {categorical_features}")

# 1. Cargar una muestra para identificar qué columnas REALES tenemos
sample_df = pd.read_parquet("data/ibm/val_step_1_R128.parquet")

# 2. Definir target y filtrar características
target = 'Winner'

# Solo tomamos las columnas que existen en el archivo y no son el target
features_reales = sample_df.drop(columns=[target]).columns.tolist()

# Clasificamos basándonos en el archivo físico
numerical_features = sample_df[features_reales].select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = sample_df[features_reales].select_dtypes(include=['object', 'category']).columns.tolist()

print("--- Clasificación Sincronizada ---")
print(f"Columnas detectadas en Parquet: {len(features_reales)}")
print(f"Numéricas: {len(numerical_features)}")
print(f"Categóricas: {len(categorical_features)}")

# 3. Pipelines y ColumnTransformer (igual que antes)
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_features),
    ('cat', cat_pipeline, categorical_features),
], verbose_feature_names_out=False)

# 1. Definir los mejores parámetros obtenidos de Optuna
# Eliminamos el prefijo 'clf__' para pasarlos directo al modelo
best_params_xgb = {
    'n_estimators': 20000,
    'learning_rate': 0.01851697194041473,
    'max_depth': 4,
    'subsample': 0.789557486760989,
    'colsample_bytree': 0.8996036961729756,
    'gamma': 2.63824665487096,
    'reg_alpha': 9.231197315531594e-05,
    'reg_lambda': 9.618812169266158e-06,
    'random_state': 16,  # Tu RND fijo
    'n_jobs': -1         # Usar todos los cores de tu Omen 16
}


rondas = ['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'F']
historico_resultados = []

print(f"--- Evaluación Física: XGBoost vs Roland Garros 2024 ---")

for i, nombre in enumerate(rondas):
    # 1. Cargar datasets del disco (WSL)
    df_train = pd.read_parquet(f"data/ibm/train_step_{i+1}_{nombre}.parquet")
    df_val = pd.read_parquet(f"data/ibm/val_step_{i+1}_{nombre}.parquet")
    
    # 2. Separar X e y
    X_train_raw = df_train.drop(columns=['Winner'])
    y_train_step = df_train['Winner']
    X_val_raw = df_val.drop(columns=['Winner'])
    y_val_step = df_val['Winner']
    
    # 3. APLICAR PIPELINES (Soluciona el error de 'object')
    # Ajustamos el preprocesador solo con los datos de entrenamiento de este paso
    X_train_step = preprocessor.fit_transform(X_train_raw)
    # Transformamos la validación usando el ajuste anterior
    X_val_step = preprocessor.transform(X_val_raw)
    
    # 4. Entrenar modelo con los parámetros de Optuna
    model = XGBClassifier(**best_params_xgb)
    model.fit(X_train_step, y_train_step)
    
    # 5. Predicción y Score
    preds = model.predict(X_val_step)
    acc = accuracy_score(y_val_step, preds)
    
    historico_resultados.append({
        'Paso': i+1,
        'Ronda': nombre,
        'Partidos': len(y_val_step),
        'Accuracy': round(acc, 4)
    })
    
    print(f"Paso {i+1} [{nombre}]: Accuracy {acc:.4f} en {len(y_val_step)} partidos.")

# Ver tabla final
resumen_final = pd.DataFrame(historico_resultados)
display(resumen_final)

--- Clasificación de Variables ---
Target: Winner
Numéricas (46): ['draw_size', 'best_of', 'tourney_points', 'diff_elo_blend', 'diff_days_since', 'diff_injury', 'diff_seed', 'diff_ht', 'diff_age', 'diff_rank', 'diff_rank_points', 'diff_h2h', 'diff_is_seeded', 'diff_is_first_match', 'diff_df_pct_last_1', 'diff_win_rate_last_1', 'diff_df_pct_last_10', 'diff_tb_rate_last_1', 'diff_win_rate_last_10', 'diff_tb_won_pct_last_5', 'diff_df_pct_last_5', 'diff_win_rate_lifetime', 'diff_win_rate_last_5', 'diff_tb_won_pct_last_10', 'diff_bp_save_pct_last_1', 'diff_1st_won_pct_last_5', 'diff_1st_won_pct_lifetime', 'diff_bp_save_pct_last_5', 'diff_bp_save_pct_lifetime', 'diff_ace_pct_last_5', 'diff_tb_rate_last_10', 'diff_ace_pct_last_1', 'diff_tb_rate_lifetime', 'diff_is_rookie', 'diff_tb_rate_last_5', 'diff_ace_pct_last_10', 'diff_tb_won_pct_last_1', 'diff_bp_save_pct_last_10', 'diff_1st_won_pct_last_1', 'diff_1st_won_pct_last_10', 'diff_elo', 'diff_elo_surface', 'year', 'month', 'day', 'round_orde

Paso 1 [R128]: Accuracy 0.7969 en 64 partidos.
Paso 2 [R64]: Accuracy 0.7931 en 29 partidos.
Paso 3 [R32]: Accuracy 0.8750 en 16 partidos.
Paso 4 [R16]: Accuracy 0.7500 en 8 partidos.
Paso 5 [QF]: Accuracy 1.0000 en 3 partidos.
Paso 6 [SF]: Accuracy 0.5000 en 2 partidos.
Paso 7 [F]: Accuracy 1.0000 en 1 partidos.


,Paso,Ronda,Partidos,Accuracy
0,1,R128,64,0.7969
1,2,R64,29,0.7931
2,3,R32,16,0.8750
3,4,R16,8,0.7500
4,5,QF,3,1.0000
5,6,SF,2,0.5000
6,7,F,1,1.0000


In [20]:
# 1. Calcular métricas globales
total_partidos = resumen_final['Partidos'].sum()
# Calculamos la media ponderada (Aciertos totales / Partidos totales)
accuracy_global = (resumen_final['Accuracy'] * resumen_final['Partidos']).sum() / total_partidos

# 2. Crear fila de total
fila_total = pd.DataFrame({
    'Paso': ['-'],
    'Ronda': ['TOTAL GLOBAL'],
    'Partidos': [total_partidos],
    'Accuracy': [round(accuracy_global, 4)]
})

# 3. Combinar y aplicar estilo
df_bonito = pd.concat([resumen_final, fila_total], ignore_index=True)

# 4. Estilizar con gradientes y formato
style_resumen = df_bonito.style.background_gradient(
    cmap='RdYlGn', # Rojo a Verde
    subset=['Accuracy'],
    vmin=0.5, vmax=0.8
).format({
    'Accuracy': "{:.2%}" # Mostrar como porcentaje: 70.54%
}).set_properties(**{
    'text-align': 'center',
    'font-weight': 'bold'
}, subset=pd.IndexSlice[df_bonito.index[-1], :]) # Negrita solo para la última fila

display(style_resumen)

# Print final informativo
print(f"\n🎯 Rendimiento final en Roland Garros: {accuracy_global:.2%}")
print(f"📦 Total de partidos evaluados: {total_partidos}")

,Paso,Ronda,Partidos,Accuracy
0,1,R128,64,79.69%
1,2,R64,29,79.31%
2,3,R32,16,87.50%
3,4,R16,8,75.00%
4,5,QF,3,100.00%
5,6,SF,2,50.00%
6,7,F,1,100.00%
7,-,TOTAL GLOBAL,123,80.49%



🎯 Rendimiento final en Roland Garros: 80.49%
📦 Total de partidos evaluados: 123


# Tiempo total de ejecución

In [19]:
print(f"Total Running time {timedelta(seconds=(time() - init_time))}")

Total Running time 0:02:08.341289
